In [184]:
import numpy as np
import pandas as pd
import math
from numpy.linalg import inv
#hello this is khai 

In [185]:
# Read the Excel file
#basecase = pd.ExcelFile('C:\\Users\\jingy\\Documents\\EE454Lab\\system_basecase.xlsx')
# khai's file location
basecase = pd.ExcelFile("C:\\Users\\kayla\\OneDrive - UW\\EE 454\\myrepo\\EE454Project\\system_basecase.xlsx")

# Read the busdata sheet
busdata = pd.read_excel(basecase, 'BusData')
# Read the linedata sheet
linedata = pd.read_excel(basecase, 'LineData')

# Convert to numpy arrays for later use
busdata_np = np.array(busdata)
linedata_np = np.array(linedata)

# # print data to test if it reads correctly
#print("Bus Data:", busdata_np.shape)
pd.DataFrame(busdata_np)




,0,1,2,3,4,5
0,1,0.0,0.0,S,0,1.04
1,2,20.1,12.5,D,0,1.0
2,3,45.2,16.0,G,42,1.025
3,4,60.8,10.9,D,0,1.0
4,5,7.6,-1.6,D,0,1.0
5,6,13.5,8.5,D,0,1.0
6,7,33.5,11.6,D,0,1.0
7,8,9.0,5.8,D,0,1.0
8,9,4.3,2.1,G,23,1.02
9,10,5.2,1.6,G,27,1.02


In [186]:
num_buses = len(busdata_np)

In [187]:
print("Line Data:", linedata_np.shape)
pd.DataFrame(linedata_np)

Line Data: (17, 6)


,0,1,2,3,4,5
0,1.0,2.0,0.01938,0.05917,0.0528,95.0
1,1.0,5.0,0.05203,0.20304,0.0492,100.0
2,2.0,3.0,0.04699,0.19797,0.0438,9999.0
3,2.0,4.0,0.05811,0.17632,0.0340,9999.0
4,2.0,5.0,0.05695,0.07388,0.0346,9999.0
5,3.0,4.0,0.67010,0.17103,0.0128,9999.0
6,4.0,5.0,0.01335,0.04211,0.0391,9999.0
7,4.0,7.0,0.00000,0.55618,0.0000,9999.0
8,5.0,6.0,0.00000,0.25202,0.0000,9999.0
9,6.0,9.0,0.09498,0.19890,0.0176,9999.0


In [188]:
def create_Y_matrix(busdata, linedata):

    # Initialize the admittance matrix Y with all zeros
    Y = np.zeros((num_buses, num_buses), dtype=complex)

    # Process each line
    for line in linedata:
        # Get the bus number also take account the 0 index based numbering
        from_bus = int(line[0]) - 1
        to_bus = int(line[1]) - 1
        R = line[2]
        X = line[3]
        # Get the half-line shunt susceptance
        B_shunt = line[4] / 2

        Z = complex(R, X)
        # Get the line admittance
        Y_line = 1 / Z

        # Add each admittance component to the matrix
        Y[from_bus, from_bus] += Y_line + complex(0, B_shunt)
        Y[to_bus, to_bus] += Y_line + complex(0, B_shunt)
        Y[from_bus, to_bus] -= Y_line
        Y[to_bus, from_bus] -= Y_line

    # (commented out as optional) 
    # Get the conductance (G) and susceptance (B) seperately
#     G = np.real(Y)
#     B = np.imag(Y)

    return Y

In [189]:
# Get the admittence matrix
Y = create_Y_matrix(busdata_np, linedata_np)
G = np.real(Y)
B = np.imag(Y)
print("Admittance Matrix:\n")

pd.DataFrame(B)


Admittance Matrix:



,0,1,2,3,4,5,6,7,8,9,10,11
0,-19.833736,15.263087,0.000000,0.000000,4.621650,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,15.263087,-33.568623,4.781863,5.115838,8.490435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,4.781863,-5.111153,0.357590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,5.115838,0.357590,-28.807011,21.578554,0.000000,1.797979,0.000000,0.000000,0.000000,0.000000,0.000000
4,4.621650,8.490435,0.000000,21.578554,-38.597128,3.967939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,3.967939,-17.303083,0.000000,0.000000,4.094074,3.175964,6.102755,0.000000
6,0.000000,0.000000,0.000000,1.797979,0.000000,0.000000,-15.155774,10.365394,0.000000,0.000000,0.000000,3.029050
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.365394,-14.734838,4.402944,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,4.094074,0.000000,4.402944,-8.475218,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,3.175964,0.000000,0.000000,0.000000,-5.405089,2.251975,0.000000


In [190]:
V_mag = busdata_np[:, 5]
V_ang = np.zeros(num_buses) #radians
busnum = busdata_np[:,0]
P_Load = busdata_np[:, 1]
Q_Load = busdata_np[:, 2]
P_Gen = busdata_np[:, 4]
bus_type = busdata_np[:, 3]

slack = None
PQind = []
PVind = []

for i in range(num_buses):


    # initialize V based on bus type
    if bus_type[i] == 'S':
        slack = i
    elif bus_type[i] == 'G':
        PVind.append(i)
    elif bus_type[i] == 'D':
        PQind.append(i)

In [191]:
print("Vmag:" + str(V_mag))
print("Vang:" + str(V_ang))

Vmag:[1.04 1.0 1.025 1.0 1.0 1.0 1.0 1.0 1.02 1.02 1.0 1.03]
Vang:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [192]:
#formulas in textbook are written as 1 being Slack, 2 to m being PV (gen), and m+1 to N being PQ 

def Pmismatches (vpu, G, B, theta, pgen, pload):
    
    #initialize array
    Pmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        
        #initialize a sum for all buses
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.cos(theta[k]-theta[i])+B[k, i]*
                                   math.sin((theta[k]-theta[i]))) 
            
        #add sum to load-gen (divided by 100 M for per-unit value) for each bus
        Pmis[k] = sum + pload[k]/100. - pgen[k]/100.
        
    return Pmis


def Qmismatches (vpu, G, B, theta, qload):
    #initialize array

    Qmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        sum = 0
        
        #initialize a sum for all buses

        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                   math.cos((theta[k]-theta[i])))
        
        #add sum to load (divided by 100 M for per-unit value) for each bus

        Qmis[k] = sum  + qload[k]/100.
        
    return Qmis


    

In [193]:
print(Pmismatches(V_mag, G, B, V_ang, P_Gen, P_Load))
print(Qmismatches(V_mag, G, B, V_ang, Q_Load))

[ 0.25723161 -0.02734074  0.09698669  0.57297383  0.02862718  0.06538008
  0.29227984  0.0523823  -0.10874737 -0.13609416  0.05110968 -0.10186511]
[ 0.77204344 -0.68767004  0.2619658   0.05711025 -0.262316   -0.09805077
 -0.01152151 -0.06355887  0.17165845  0.10295681 -0.0994884   0.1789003 ]


In [194]:
#mostly done ? 
def HMatrix (G, B, vpu, theta):
    #HMatrix will iterate P(row) from buses 2 to N and theta(col) from buses 2 to N
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = np.arange(1, num_buses, dtype=int)
  
    
    #empty H array
    H = np.zeros((len(rows), len(cols)))
    
    #r represents the 
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        matchBus = False
        matchCol = None
        
        for i in range(0, num_buses):
            

            
            
            
            if (i != k and i in cols):
                H[r,c] = vpu[k]*vpu[i]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
#                 print("H has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(-1*G[k, i]*math.sin((theta[k]-theta[i]))+B[k, i]*
                                            math.cos((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
            
                
        if (matchBus):     
        
            H[r,matchCol] = sum
        
        r += 1
        
    return H

In [195]:
def MMatrix (G, B, vpu, theta):
    #MMatrix will iterate P(row) from buses 2 to N and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = PQind
    
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty M array
    M = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            
            #print("i is " + str(i) + "and mcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                M[r,c] = vpu[k]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
#                 print("M has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
                
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            M[r,matchCol] = 2*G[k, k]*vpu[k] + sum
        
        r += 1
        
    return M

In [196]:
def NMatrix (G, B, vpu, theta):
    #NMatrix will iterate Q(row) for PQ buses and theta(col) for PQ and PV buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = np.arange(1, num_buses, dtype=int)
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty N array
    N = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
#         print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
#             print("i is " + str(i) + "and Ncol is " + str(c))

            
            
            
            if (i != k and i in cols):
                N[r,c] = vpu[k]*vpu[i]*(-1*G[k, i]*math.cos((theta[k]-theta[i]))-B[k, i]*
                                            math.sin((theta[k]-theta[i])))
#                 print("N has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c     
            
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(G[k, i]*math.cos((theta[k]-theta[i]))+B[k, i]*
                                            math.sin((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            N[r,matchCol] = sum
        
        r += 1
        
    return N

In [197]:
def LMatrix (G, B, vpu, theta):
    #LMatrix will iterate Q(row) for PQ buses and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = PQind
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty L array
    L = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            #print("i is " + str(i) + "and lcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                L[r,c] = vpu[k]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
#                 print("L has been printed at " + str(r) + " " + str(c))

            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                            math.cos((theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            L[r,matchCol] =  -2*B[k, k]*vpu[k] + sum
        
        r += 1
        
    return L

In [202]:
def build_jacobian(G, B, vpu, theta):
    mat1 = np.concatenate((HMatrix(G, B, vpu, theta),NMatrix(G, B, vpu, theta)), axis=0)
    mat2 = np.concatenate((MMatrix(G, B, vpu, theta),LMatrix(G, B, vpu, theta)), axis=0)

    J = np.concatenate((mat1, mat2), axis=1)
    return J

In [203]:
J = build_jacobian(G, B, V_mag, V_ang)
pd.DataFrame(J)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,34.381293,-4.901410,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.136649,-1.686033,-6.544806,0.000000,0.000000,0.000000,0.000000
1,-4.901410,5.267939,-0.366530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.163395,-1.436073,0.000000,0.000000,0.000000,0.000000,0.000000
2,-5.115838,-0.366530,28.858901,-21.578554,0.000000,-1.797979,0.000000,0.000000,0.000000,0.000000,0.000000,-1.686033,9.893034,-6.840981,0.000000,0.000000,0.000000,0.000000
3,-8.490435,0.000000,-21.578554,38.843444,-3.967939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.544806,-6.840981,14.522734,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,-3.967939,17.486134,0.000000,0.000000,-4.175956,-3.239483,-6.102755,0.000000,0.000000,0.000000,0.000000,6.510303,0.000000,0.000000,-3.098927
5,0.000000,0.000000,-1.797979,0.000000,0.000000,15.283295,-10.365394,0.000000,0.000000,0.000000,-3.119922,0.000000,0.000000,0.000000,0.000000,5.283335,-3.902050,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,-10.365394,14.856397,-4.491003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.902050,5.745317,0.000000
7,0.000000,0.000000,0.000000,0.000000,-4.175956,0.000000,-4.491003,8.666958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.994129,0.000000,-1.918502,0.000000
8,0.000000,0.000000,0.000000,0.000000,-3.239483,0.000000,0.000000,0.000000,5.536497,-2.297014,0.000000,0.000000,0.000000,0.000000,-1.556487,0.000000,0.000000,-2.538805
9,0.000000,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,0.000000,-2.297014,10.784182,-2.384412,0.000000,0.000000,0.000000,-3.098927,0.000000,0.000000,6.641056


In [205]:
print(PQind)

[1, 3, 4, 5, 6, 7, 10]


In [234]:
def NR (vpu, G, B, theta, pgen, pload, qload):
    
    twoToN =  np.arange(1, num_buses, dtype=int)
    
    allPmis = Pmismatches(vpu, G, B, theta, pgen, pload)
    
    allQmis =  Qmismatches(vpu, G, B, theta, qload)
    
    newTheta = np.copy(theta)
    newV = np.copy(vpu)
    
    
#     firstPMismatch = allPmis[twoToN]
#     firstQMismatch = allQmis[PQind]
    
    
    
    currentPMismatch = allPmis
    currentQMismatch = allQmis
    
    print("P mismatch is " + str(currentPMismatch[twoToN]))
    print("Q mismatch is " + str(currentQMismatch[PQind]))

    notSmall = True

    while(notSmall):
        J = build_jacobian(G, B, newV, newTheta)
        delta = np.matmul(inv(J), np.concatenate((currentPMismatch[twoToN], currentQMismatch[PQind])))

#         print("delta is " + str(delta))
# #         print(delta[0:len(twoToN)])
# #         print(delta[len(twoToN):])


#         print("newV is " + str(newV) )

#         print("newT is " + str(newTheta))
        
        newTheta[twoToN] = newTheta[twoToN]-(delta[0:len(twoToN)])

        newV[PQind] = newV[PQind] - delta[len(twoToN):]

        
        currentPMismatch = Pmismatches(newV, G, B, newTheta, pgen, pload)
        currentQMismatch = Qmismatches(newV, G, B, newTheta, qload)

        print("P mismatch is " + str(currentPMismatch[twoToN]))
        print("Q mismatch is " + str(currentQMismatch[PQind]))
#         print("newV is " + str(newV) )

#         print("newT is " + str(newTheta) )

        if(np.all(np.abs(currentPMismatch[twoToN]) <= .001) and np.all(np.abs(currentQMismatch[PQind]) <= .001)):
            notSmall = False
            

    
    return newV, newTheta
        

In [235]:
newV, newTheta = NR(V_mag, G, B, V_ang, P_Gen, P_Load, Q_Load)

P mismatch is [-0.02734074  0.09698669  0.57297383  0.02862718  0.06538008  0.29227984
  0.0523823  -0.10874737 -0.13609416  0.05110968 -0.10186511]
Q mismatch is [-0.68767004  0.05711025 -0.262316   -0.09805077 -0.01152151 -0.06355887
 -0.0994884 ]
P mismatch is [ 0.01133931 -0.00117994 -0.01159527  0.00270448 -0.00043319  0.00246522
  0.00081176  0.00057529  0.00356329  0.00084786  0.00028913]
Q mismatch is [0.03437284 0.01099777 0.01991981 0.00273756 0.0037804  0.00276784
 0.0020708 ]
P mismatch is [ 3.64771495e-05  1.12056583e-05 -4.64315540e-05  1.49792516e-05
  1.24662650e-05  1.03116194e-05  1.92455380e-06  3.00183277e-06
  4.70142105e-06  2.82492694e-06  2.38799586e-06]
Q mismatch is [6.45525042e-05 3.84532046e-05 4.94129761e-05 8.57415795e-06
 7.76467056e-06 2.62413305e-06 1.37945419e-06]


In [227]:
pd.DataFrame(newV)

,0
0,1.04
1,1.023548
2,1.025
3,1.003778
4,1.012579
5,1.009502
6,0.993517
7,0.997423
8,1.02
9,1.02


In [228]:
pd.DataFrame(newTheta)

,0
0,0.000000
1,-0.040128
2,-0.054542
3,-0.075502
4,-0.058795
5,-0.070665
6,-0.116411
7,-0.108529
8,-0.076169
9,-0.041617


In [236]:
def injectedP(vpu, theta, G, B):
    #initialize array
    Pmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        
        #initialize a sum for all buses
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.cos(theta[k]-theta[i])+B[k, i]*
                                   math.sin((theta[k]-theta[i]))) 
            
        #add sum to load-gen (divided by 100 M for per-unit value) for each bus
        Pmis[k] = sum 
        
    return Pmis

In [237]:
def injectedQ(vpu, theta, G, B):
    #initialize array

    Qmis = np.zeros(num_buses)
    
    #for each bus...
    for k in range(num_buses):
        sum = 0
        
        #initialize a sum for all buses

        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.sin((theta[k]-theta[i]))-B[k, i]*
                                   math.cos((theta[k]-theta[i])))
        
        #add sum to load (divided by 100 M for per-unit value) for each bus

        Qmis[k] = sum  
        
    return Qmis

In [244]:
pd.DataFrame(100*injectedQ(newV, newTheta, G, B))
# print(100*injectedQ(newV, newTheta, G, B))

,0
0,7.250484
1,-12.493545
2,-2.728565
3,-10.896155
4,1.604941
5,-8.499143
6,-11.599224
7,-5.799738
8,7.412923
9,-8.379973
